In [16]:
!MPLCONFIGDIR=/scratch/tsoies-DL/tmp
import pandas as pd
import numpy as np
import os
import pysam
from tqdm import tqdm
import argparse
import seaborn as sns
import matplotlib.pyplot as plt
import csv
from itertools import chain
from collections import defaultdict, Counter
from multiprocessing import Pool
import sys
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pandarallel import pandarallel

In [15]:
set([x for xs in [['a', 'bc'], ['c', 'd'], 'ep'] for x in xs if type(xs) == list])

{'a', 'bc', 'c', 'd'}

In [35]:
with open('/scratch/tsoies-DL/data/E250088063_L01_MERGED.fq') as fastq:
    for record in SeqIO.parse(fastq, 'fastq'):
        print(''.join([chr(i+33) for i in record.letter_annotations['phred_quality']]))

7BACDCA:>===>>DC@D?BA=B8=:A>B@
A;4@AB@D?=@9=?@;>@AAB<@9BDDDCC
GGGGGGGGGGGGGGGGGGGGGGGGGGGGFG
@>>>?=A:=C>=<>A@AA:=BAA>>A;B@@
FFDEDCEFDBFEGGADEGFFGFCAFGFGED
FFFEGGGFFGGGFFFFGFFFEFFGFEFFEF
GEGGGGGGGFGGGFGGFGFGGGGDFGGGFG
GFFGGGGFGGGFGFGGFFGGC:GGGFEGGG
FFDEFGEGFEFEEFGDEFFEGFGFBGBEFF
GGFGGGGGFGGEGFFGGFFFFFFGGGEFGF
EFEDGGGFFDDFCEGEGFGGGDFFFGGFGG
BE?@AB?BB@AD@@DD@?EDE?@BADECEC
F:ECBGFEGFBGFE"EFGDEFCED?FF>FE
FEGFCGFFEGGEGFEDGGFFCGG>GGFFEF
FEGGFGGEGEFFFFEGAFGGFEFFGFFBGF
FFBGFFFEFDFF@F;F=EFGG9DFDEFFAE
FGFGGGFGFGGEFGFGGGGFGFGGFGGGGG
GGGEGGGGEGGGGGGGFGGGGGGDGGGGGG
?CC?BDBCAD=@@A?C>@B?9BA@6@4@@C
GGGGGGFGGGFGGGGGGGGGGFGGGGGFGG
BGCGGFCDFDFFGGGGGFGGFDFGGGGGGF
EEEEFFFGFDFFFGFGFCFFFFFFFGEFGG
FAFF?FFF=EEEFDDICEEDABEFFEEEDE
GFGFGGEGGGGGGGGGGGGG@GF:GGGGGE
FGFGFFGGGGGGGGGFGGGGEGGGGGGFFG
GGFEGGFFFEGGGGGGDFEGGGGEFGFGFD
EFEFEFEBFCFDGGFDDFGFGGFF>DGGGF
D<DDCCFC:CCFGCE1DEFB>>EF=?:E/D
EFFFEDFGFGFGFGGGFGEFGDFGGGG?FF
GGGGFGGGGGGGGGGGGGFGGGGGGGFGGG
FF?DDBFDF>FEGED@EBE@FFCDEDFGAE
AFD;C@FFFCGEFFCBFCEDFADFGFFFFG
GGEFGGGF

EGGGGGGGFGGGGGGGGGGGGGGGGGGGFF
B?@@@@?A>B83@A9=8=<?=5=B==>:,7
:AA>=<=>@A<@?@?;>3A@;=?<=?=>>?
FFEFDFGFGGGEGBDFGFFGGGFGDFEGGE
FGGGFFGFFEGGGFEGFFGGFDGEGGGGFG
FFBGGEFFFGECFEGCFFEGFFGFGG<AGE
FFFFGGGGGGCGEFGGGGFG?DFFFG0GGF
GFFFFFEFFFFFFFFFGDGCFFFGGGFGGE
F>EBCCB@C@FCCBB?@;CFDEA@?>ECEF
DHG"GGGFBFGBGGGBFBFFFG"GFBFEBE
GFGFFFFDEEGEGFFFGEGEGGGGGGFFFF
=ECDCBE@FFEFEGDEFEFEGFFACEFFFG
FFGGGFGGGGGGFGFFGGGEGFGGGGFGGG
5DCCBCBD?AABCBA@9D3DBECFC@A@;A
:@>=?;6=?A=C>;;?BAA>?==ABC6?9@
FDD2BECEBF=9D7CFF8=:FE?;=79E=.
FEGEFFFGGDFFEFGEFGDCDEFFFGEDED
FFEGFFFGGGFGGGGFGFFGGGFEGGGEFG
G7GGGFGGFGGEEGGFGGGGGFGGFG@GAG
EEDDEC?C@EE@CFEEFEDGF:EDEC>E<D
EFGFDEEEFFGFGGFCFCEFFGEGGGGBEE
"FGFBFGEF"EEFBFBIFC>8FBF@D>FF>
GGGGGFEFFGFFFGGGGGGEFGGGGGFFFG
GGGCEGDAEGGEGGGGFEEEDCGE8=GAGC
CGCCAAD>@DDDABFACEGEFEBFCFFFFF
<;@>@=:@C3A,4>-<@<?@=@<>AC??70
DGGEFGEFGFGG:GCFGGGEGFFEGGGGGF
BGFGGGGGFFFGGGGGGGGGGGGGGGGGGG
BCDA@DAB>3ECCB==?=>>=CCD/CCA?>
DFDEEEGF@GGGDGFGFFFDFEFGGFGCFF
EFFEFEFCDFEFFEFDEFGEGFFD@FGGFC
ECBDECCADCEAA9>DDD=?DDEFFE6BBE
GEGBEFFF

KeyboardInterrupt: 

In [3]:
barcodes = pd.read_csv('/scratch/tsoies-DL/data/barcodes_count1000.csv')

In [12]:
lisT = barcodes['barcode'].to_list()
seT = set(barcodes['barcode'].to_list())
sample = barcodes['barcode'].sample(100).to_list()

In [19]:
%%timeit
for barcode in sample:
    i = barcode in seT

2.41 μs ± 3.51 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [20]:
barcode2M_stat = pd.read_csv('/scratch/tsoies-DL/LOCAL_MixedGenomeAlignment_out/M_stats_df.csv')['M_statistics']

In [24]:
len(barcode2M_stat[barcode2M_stat.reset_index()['M_statistics']>0.75])

12166

In [2]:
barcode2count = pd.read_csv('/datasets/tsoies-DL/chaika_data/single_cell_1A_cDNA/barcode2count.csv')
barcode2M_stat = pd.read_csv('/datasets/tsoies-DL/chaika_data/single_cell_1A_cDNA/barcode2M_stat.csv')[['barcode', 'M']]

In [6]:
barcode2sp = pd.read_csv('/scratch/tsoies-DL/data/barcode2sp.csv', index_col=0)

In [ ]:
def write_barcode_lists(row, out_path):
    filename = row.values[0] + '_barcodes.csv'
    barcodes = pd.DataFrame({'barcode': row.values[1]})
    out_file = os.path.join(out_path, filename)
    barcodes.to_csv(out_file, header=False, index=False)
    

In [7]:
sp2barcodes = barcode2sp.groupby(by='species').agg(lambda x: ','.join(x).split(',')).reset_index()

In [16]:
sp2barcodes

,species,barcode
0,Acomys_dimidiatus,"[CTTCAATAATCCTGTTAATG, GCTTAATCAAAAGCCTTGAC, T..."
1,Carassius_gibelio,"[TGACTGTCAGGTCGCCTCTT, GTGTTATCCGCGCGAGTCGA, G..."
2,Dicrostonyx_torquatus,"[CTACGTAGGTGACGGACAGC, AGTTGTTCGTCGTTGGTGGT, G..."
3,Ellobius_talpinus,"[GGTGGAAGGTGGTAGGATAC, AATGCCAGTTAGTCGACCAT, T..."
4,Larus_michahellis,"[GATAATACGCTATAGAAGCT, AATCCGCACTCCTCGCCTCT, A..."
5,Myotis_brandtii,"[CGACCTTGGTGTTGTTCTCT, ACATAGGTGGTAGGACAGCC, T..."
6,Panthera_tigris,"[TGTCCGCGATAACCATATAC, TATCTTGGTCTCACGCCGCG, C..."
7,Sander_lucioperca,"[TACGATGCACGCCTGTGTTA, CTGAACCACATGACTTCGTA, G..."
8,Ursus_maritimus,"[AACAGACGACCCAGAGTTGA, TTGATCTTAAATCTCCAGAA, C..."


In [ ]:
def make_barcode2readName_dict() -> dict:
    print('********** make_barcode2readName_dict **********', flush=True)
    barcode2readName_dict = {}
    with open('/scratch/tsoies-DL/data/E250088063_L01_merged.csv', mode='r') as infile:
        reader = csv.reader(infile)
        for rows in reader:
            read_name = rows[0]
            barcode = rows[1][0:20]
            barcode2readName_dict[barcode] = read_name
    print('[make_barcode2readName_dict]: DONE', flush=True)
    return barcode2readName_dict

In [14]:
barcode2readName = make_barcode2readName_dict()

********** make_barcode2readName_dict **********
[make_barcode2readName_dict]: DONE


In [ ]:
sp2barcodes['barcode'] = sp2barcodes['barcode'].apply(lambda x: [barcode2readName[i] for i in x])


In [6]:
sp2barcodes

NameError: name 'sp2barcodes' is not defined

In [11]:
def A(row):
    print(row.iloc[0])
    print(row.iloc[1])

In [12]:
sp2barcodes.apply(A, axis=1)

Acomys_dimidiatus
['CTTCAATAATCCTGTTAATG', 'GCTTAATCAAAAGCCTTGAC', 'TGTCCGCGATCTTGTTGCTA', 'GGATACTGATCCAGATATCT', 'TGCACTTATCCGGCGTGTAA', 'AACGAGTATTAACGAGTATT', 'TTCGTTGTGCTGATGTCGAT', 'TCTACGGTAATTGAGGAGAC', 'AAGGTTCAATTTACGTTCAG', 'CTATGAACGAATTACATAAT', 'GACGAAGACTACCGATGACG', 'GAGTAGTTCAGCCGAATAGT', 'AAGTAAGCGTCGCGATACCT', 'CACAGAAGCCAGGCAGTTCC', 'AATGCGCCACCGAGGTTAAC', 'ACGCATAGTGAACGGTCTTG', 'AGCCGCACCGAGTTGTTCGT', 'AGATTAAGGAGCGTATAATC', 'AGATCGCCTAGCGTTAACGC', 'TGACGTCCTTTATAGGCTGT', 'TTATTATCAGATTCATCAAG', 'ACGGATCAGCCCATGCTCAG', 'AGTGGACACCGATTACAATG', 'GCCGTGATCTCTGTGCTAGG', 'CTGGCTTCCGTATCATCTCT', 'TATGCCTGATTGGTATTGTC', 'AAGGATCCGAAATACGTTGG', 'TCTCGCCGATGGCGTCAGTA', 'CGCGATACCTGTATATGAAG', 'GCACTTCACGTTCGTCGCCA', 'CCACTGGATTGCTTGACAAT', 'GCGGAGCCATGCGTTAGATG', 'AGAGGAGGCGCAGTGTATGC', 'CTTCTACGATAACAGCATAG', 'CGTATGAATTTTACTGCTGA', 'GACTGTATGGTAGTTGTCAC', 'AACGAGTATTGCCATGTGTC', 'ATGCACTGGTCGACCTTGGT', 'TAAGGCTCGGCCATAAGGAG', 'ATGGAACAATATTCACGGTG', 'GACTTGAGGTAACCTGGTGA

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
dtype: object